In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
sys.path.insert(0, "C:/workspace/data-science//libs")
from simulation_core import Simulation

pd.set_option('display.max_columns', None)

86400 24 3600


The only plot function  left here, all other plotting functions related to the simulations are placed in the ```Simulation``` class of the ```simulation_core.py```

In [ ]:
def plot_mints(mitigated_mints: pd.DataFrame, pure_mints: pd.DataFrame, pool_name: str, width: int=10, height: int=10):
    """plot red line of original mints and blue dots of mitigated ones

    Args:
        mitigated_mints (pd.DataFrame): dataset of mitigated mints
        pure_mints (pd.DataFrame): pure dataset of mints without mitigation
        width (int, optional): width. Defaults to 10.
        height (int, optional): height. Defaults to 10.
    """
    fig, ax = plt.subplots(figsize=(width, height))
    ax2 = ax.twinx()

    ax.plot(pd.to_datetime(pure_mints.timestamp, unit='s'), pure_mints.Y_amount, c='red', label='mitigation off')
    ax2.plot_date(pd.to_datetime(mitigated_mints.timestamp, unit='s'), mitigated_mints.Y_amount, linestyle='--', c='blue', label='mitigation on')

    ax.set_title(f'Scatterplot of the mints for {pool_name} pool')
    plt.legend()
    ax.set_xlabel('time')
    ax.set_ylabel('Y token amount')
    ax2.set_ylabel('Y token mitigated amount')
    fig.autofmt_xdate(rotation=25)
    ax.grid(True, linestyle='--')

    plt.show()

Setting up the simulations parameters, with identifying their paths

**cell below is executed ONLY ONCE BECAUSE OF FOUND MINTS TABLE ERROR**

In some unique cases pools set pairs in unexpected order of tokens, meaning that all mints and burns will differ from expected format and therefore it is required to set them back into expected order. Unexpected order can be found if during simulations most of the transactions are blocked by ```NOT_ENOUGH_RESERVES``` status.

In [ ]:
# mints_df = pd.read_pickle(f'{os.getcwd()}\data\pair_history\PERL_WETH\perl_weth_mints.pkl')
# perl_series = mints_df.amount1.copy()   # copy placed, because otherwise reference is placed
# weth_series = mints_df.amount0.copy()
# mints_df.amount0 = perl_series
# mints_df.amount1 = weth_series
# mints_df.to_pickle(f'{os.getcwd()}\data\pair_history\PERL_WETH\perl_weth_mints.pkl')

Each pool has its simulation parameters required to be set up in order to perform correct reading of the pickle files and where to save all simulation results.

In [ ]:
# Meme token pools
doge_weth_sim = Simulation(experiment_id = 21, x_name = 'DOGE', y_name = 'WETH', window_size = 24, 
                           base_dir = fr"{os.getcwd()}\data\real_transactions\experiment_")
elon_weth_sim = Simulation(experiment_id = 22, x_name = 'ELON', y_name = 'WETH', window_size = 24, 
                           base_dir = fr"{os.getcwd()}\data\real_transactions\experiment_")
shib_weth_sim = Simulation(experiment_id = 23, x_name = 'SHIB', y_name = 'WETH', window_size = 24, 
                           base_dir = fr"{os.getcwd()}\data\real_transactions\experiment_")
squid_weth_sim = Simulation(experiment_id = 24, x_name = 'SQUID', y_name = 'WETH', window_size = 24, 
                           base_dir = fr"{os.getcwd()}\data\real_transactions\experiment_")

**IMPORTANT: launch those simulations only if there is no data in package ```data\real_transactions```, otherwise skip this part (comment it)**

In [ ]:
# # meme token pools
# doge_weth_sim.run_simulation(isBurnAvailable = True, isMintAvailable = True)    # passed, experiment 21
# elon_weth_sim.run_simulation(isBurnAvailable = True, isMintAvailable = True)    # passed, experiment 22
# shib_weth_sim.run_simulation(isBurnAvailable = True, isMintAvailable = True)    # passed, experiment 23
# squid_weth_sim.run_simulation(isBurnAvailable = False, isMintAvailable = True)  # passed, experiment 24

## Meme-tokens pools (DOGE/WETH, SHIB/WETH, ELON/WETH, SQUID/WETH)

In [ ]:
doge_weth_swaps_mitigation_off_df, doge_weth_mints_mitigation_off_df, doge_weth_burns_mitigation_off_df = doge_weth_sim.read_simulation(0)
doge_weth_swaps_mitigation_on_df, doge_weth_mints_mitigation_on_df, doge_weth_burns_mitigation_on_df = doge_weth_sim.read_simulation(1)

shib_weth_swaps_mitigation_off_df, shib_weth_mints_mitigation_off_df, shib_weth_burns_mitigation_off_df = shib_weth_sim.read_simulation(0)
shib_weth_swaps_mitigation_on_df, shib_weth_mints_mitigation_on_df, shib_weth_burns_mitigation_on_df = shib_weth_sim.read_simulation(1)

elon_weth_swaps_mitigation_off_df, elon_weth_mints_mitigation_off_df, elon_weth_burns_mitigation_off_df = elon_weth_sim.read_simulation(0)
elon_weth_swaps_mitigation_on_df, elon_weth_mints_mitigation_on_df, elon_weth_burns_mitigation_on_df = elon_weth_sim.read_simulation(1)

squid_weth_swaps_mitigation_off_df, squid_weth_mints_mitigation_off_df, squid_weth_burns_mitigation_off_df = squid_weth_sim.read_simulation(0)
squid_weth_swaps_mitigation_on_df, squid_weth_mints_mitigation_on_df, squid_weth_burns_mitigation_on_df = squid_weth_sim.read_simulation(1)

# DOGE/WETH

In [ ]:
doge_weth_mints_mitigation_off_df

In [ ]:
doge_weth_burns_mitigation_off_df

In [ ]:
doge_weth_swaps_mitigation_off_df

In [ ]:
plot_mints(doge_weth_mints_mitigation_on_df, doge_weth_mints_mitigation_off_df, pool_name='DOGE/WETH')

## MEV transactions extraction and analysis

### MEV attacks and possible MEVs extraction

There are 136 MEV attacks performed on the pool and 8789 possible attacks

In [ ]:
doge_weth_filtered_swaps_df, doge_weth_mevs_df = doge_weth_sim.extract_filtered_and_mevs_dfs()

In [ ]:
doge_weth_suspicious_filtered_swaps_df, doge_weth_possible_mevs_df = doge_weth_sim.extract_suspicious_and_filtered_swaps_dfs()

In [ ]:
doge_weth_mevs_df = doge_weth_sim.calculate_attack_profit(doge_weth_mevs_df)

### Plotting values of MEVs and possible MEVs with filtered swaps values

In [ ]:
doge_weth_sim.show_swaps_and_mevs_by_token(doge_weth_filtered_swaps_df, doge_weth_mevs_df, width=15, height=5)

In [ ]:
doge_weth_sim.show_swaps_and_mevs_by_token(doge_weth_suspicious_filtered_swaps_df, doge_weth_possible_mevs_df, width=15, height=5)

### Plotting MEV transactions and possible MEV transactions with filtered swaps daily count distributions

In [ ]:
doge_weth_sim.show_swaps_and_mevs_daily_count_by_token(doge_weth_filtered_swaps_df, doge_weth_mevs_df, mevs_alter_axis=True)

In [ ]:
doge_weth_sim.show_swaps_and_mevs_daily_count_by_token(doge_weth_suspicious_filtered_swaps_df, doge_weth_possible_mevs_df, mevs_alter_axis=True)

### Plotting ratio of MEV attacks to filtered swaps

In [ ]:
doge_weth_sim.show_mevs_to_swaps_ratio(doge_weth_filtered_swaps_df, doge_weth_mevs_df)

In [ ]:
doge_weth_sim.show_mevs_to_swaps_ratio(doge_weth_suspicious_filtered_swaps_df, doge_weth_possible_mevs_df)

### Plotting ratio of MEVs and suspicious transactions to filtered swaps

In [ ]:
doge_weth_sim.show_mevs_to_reserves_ratio(doge_weth_swaps_mitigation_off_df, doge_weth_mevs_df, width=15, height=10)
doge_weth_sim.show_mevs_values_to_reserves_ratio(doge_weth_swaps_mitigation_off_df, doge_weth_mevs_df, width=15, height=10)

### Plotting transaction values distribution of passed swaps, blocked swaps and not enough reserves swaps

In [ ]:
doge_weth_sim.plot_transactions_by_type(doge_weth_swaps_mitigation_on_df, ignore_success=False, width=15, height=7)

### Plotting mitigation comparison, slice factor distribution, reserves distributions, cumulative prices distributions, transaction frequency distributions, price impact after each transaction distributions

In [ ]:
doge_weth_sim.show_mitigation_comparison(doge_weth_swaps_mitigation_off_df, doge_weth_swaps_mitigation_on_df)
doge_weth_sim.px_hist_blocked_transactions_slice(doge_weth_swaps_mitigation_on_df)
doge_weth_sim.px_hist_blocked_transactions_difference_from_oracle(doge_weth_swaps_mitigation_on_df)
doge_weth_sim.plot_slice_distribution_and_diff_limit_line(doge_weth_swaps_mitigation_on_df)
doge_weth_sim.plot_reserves_with_and_without_mitigation(doge_weth_swaps_mitigation_off_df, doge_weth_swaps_mitigation_on_df)
doge_weth_sim.plot_cumulative_prices_with_and_without_mitigation(doge_weth_swaps_mitigation_off_df, doge_weth_swaps_mitigation_off_df)
doge_weth_sim.plot_price_distribution(doge_weth_swaps_mitigation_off_df, doge_weth_swaps_mitigation_on_df, separate_plots=True)
doge_weth_sim.plot_frequency_distribution(doge_weth_swaps_mitigation_on_df)
doge_weth_sim.plot_price_impact(doge_weth_swaps_mitigation_off_df, doge_weth_swaps_mitigation_on_df, 
                                smallest_y=doge_weth_swaps_mitigation_off_df['price_diff'].min(), 
                                biggest_y=doge_weth_swaps_mitigation_off_df['price_diff'].max())

### MEVs blocked by mitigation mechanism

In [ ]:
doge_weth_blocked_df = doge_weth_swaps_mitigation_on_df[doge_weth_swaps_mitigation_on_df['status'] == 'BLOCKED_BY_VOLATILITY_MITIGATION']
doge_weth_blocked_mevs_df = pd.merge(doge_weth_blocked_df, doge_weth_mevs_df, on='txd')
doge_weth_blocked_mevs_df

In [ ]:
doge_weth_blocked_df = doge_weth_swaps_mitigation_on_df[doge_weth_swaps_mitigation_on_df['status'] == 'BLOCKED_BY_VOLATILITY_MITIGATION']
doge_weth_blocked_mevs_df = pd.merge(doge_weth_blocked_df, doge_weth_possible_mevs_df, on='txd')
doge_weth_blocked_mevs_df

# SHIB/WETH

In [ ]:
shib_weth_mints_mitigation_off_df

In [ ]:
shib_weth_burns_mitigation_off_df

In [ ]:
shib_weth_swaps_mitigation_off_df

In [ ]:
plot_mints(shib_weth_mints_mitigation_on_df, shib_weth_mints_mitigation_off_df, pool_name='SHIB/WETH')

## MEV transactions extraction and analysis

### MEV attacks and possible MEVs extraction

There are 136 MEV attacks performed on the pool and 8789 possible attacks

In [ ]:
shib_weth_filtered_swaps_df, shib_weth_mevs_df = shib_weth_sim.extract_filtered_and_mevs_dfs()

In [ ]:
shib_weth_suspicious_filtered_swaps_df, shib_weth_possible_mevs_df = shib_weth_sim.extract_suspicious_and_filtered_swaps_dfs()

In [ ]:
shib_weth_mevs_df = shib_weth_sim.calculate_attack_profit(shib_weth_mevs_df)

### Plotting values of MEVs and possible MEVs with filtered swaps values

In [ ]:
shib_weth_sim.show_swaps_and_mevs_by_token(shib_weth_filtered_swaps_df, shib_weth_mevs_df, width=15, height=5)

In [ ]:
shib_weth_sim.show_swaps_and_mevs_by_token(shib_weth_suspicious_filtered_swaps_df, shib_weth_possible_mevs_df, width=15, height=5)

### Plotting MEV transactions and possible MEV transactions with filtered swaps daily count distributions

In [ ]:
shib_weth_sim.show_swaps_and_mevs_daily_count_by_token(shib_weth_filtered_swaps_df, shib_weth_mevs_df, mevs_alter_axis=True)

In [ ]:
shib_weth_sim.show_swaps_and_mevs_daily_count_by_token(shib_weth_suspicious_filtered_swaps_df, shib_weth_possible_mevs_df, mevs_alter_axis=True)

### Plotting ratio of MEV attacks to filtered swaps

In [ ]:
shib_weth_sim.show_mevs_to_swaps_ratio(shib_weth_filtered_swaps_df, shib_weth_mevs_df)

In [ ]:
shib_weth_sim.show_mevs_to_swaps_ratio(shib_weth_suspicious_filtered_swaps_df, shib_weth_possible_mevs_df)

### Plotting ratio of MEVs and suspicious transactions to filtered swaps

In [ ]:
shib_weth_sim.show_mevs_to_reserves_ratio(shib_weth_swaps_mitigation_off_df, shib_weth_mevs_df, width=15, height=10)
shib_weth_sim.show_mevs_values_to_reserves_ratio(shib_weth_swaps_mitigation_off_df, shib_weth_mevs_df, width=15, height=10)

### Plotting transaction values distribution of passed swaps, blocked swaps and not enough reserves swaps

In [ ]:
shib_weth_sim.plot_transactions_by_type(shib_weth_swaps_mitigation_on_df, ignore_success=False, width=15, height=7)

### Plotting mitigation comparison, slice factor distribution, reserves distributions, cumulative prices distributions, transaction frequency distributions, price impact after each transaction distributions

In [ ]:
shib_weth_sim.show_mitigation_comparison(shib_weth_swaps_mitigation_off_df, shib_weth_swaps_mitigation_on_df)
shib_weth_sim.px_hist_blocked_transactions_slice(shib_weth_swaps_mitigation_on_df)
shib_weth_sim.px_hist_blocked_transactions_difference_from_oracle(shib_weth_swaps_mitigation_on_df)
shib_weth_sim.plot_slice_distribution_and_diff_limit_line(shib_weth_swaps_mitigation_on_df)
shib_weth_sim.plot_reserves_with_and_without_mitigation(shib_weth_swaps_mitigation_off_df, shib_weth_swaps_mitigation_on_df)
shib_weth_sim.plot_cumulative_prices_with_and_without_mitigation(shib_weth_swaps_mitigation_off_df, shib_weth_swaps_mitigation_off_df, make_big_num_convert=True)
shib_weth_sim.plot_price_distribution(shib_weth_swaps_mitigation_off_df, shib_weth_swaps_mitigation_on_df, separate_plots=True)
shib_weth_sim.plot_frequency_distribution(shib_weth_swaps_mitigation_on_df)
shib_weth_sim.plot_price_impact(shib_weth_swaps_mitigation_off_df, shib_weth_swaps_mitigation_on_df, 
                                smallest_y=shib_weth_swaps_mitigation_off_df['price_diff'].min(), 
                                biggest_y=shib_weth_swaps_mitigation_off_df['price_diff'].max())

In [ ]:
shib_weth_blocked_df = shib_weth_swaps_mitigation_on_df[shib_weth_swaps_mitigation_on_df['status'] == 'BLOCKED_BY_VOLATILITY_MITIGATION']
shib_weth_blocked_mevs_df = pd.merge(shib_weth_blocked_df, shib_weth_mevs_df, on='txd')
shib_weth_blocked_mevs_df

In [ ]:
shib_weth_blocked_df = shib_weth_swaps_mitigation_on_df[shib_weth_swaps_mitigation_on_df['status'] == 'BLOCKED_BY_VOLATILITY_MITIGATION']
shib_weth_blocked_mevs_df = pd.merge(shib_weth_blocked_df, shib_weth_possible_mevs_df, on='txd')
shib_weth_blocked_mevs_df

# ELON/WETH

In [ ]:
elon_weth_mints_mitigation_off_df

In [ ]:
elon_weth_burns_mitigation_off_df

In [ ]:
elon_weth_swaps_mitigation_off_df

In [ ]:
plot_mints(elon_weth_mints_mitigation_on_df, elon_weth_mints_mitigation_off_df, pool_name='ELON/WETH')

## MEV transactions extraction and analysis

### MEV attacks and possible MEVs extraction

There are 136 MEV attacks performed on the pool and 8789 possible attacks

In [ ]:
elon_weth_filtered_swaps_df, elon_weth_mevs_df = elon_weth_sim.extract_filtered_and_mevs_dfs()

In [ ]:
elon_weth_suspicious_filtered_swaps_df, elon_weth_possible_mevs_df = elon_weth_sim.extract_suspicious_and_filtered_swaps_dfs()

In [ ]:
elon_weth_mevs_df = elon_weth_sim.calculate_attack_profit(elon_weth_mevs_df)

### Plotting values of MEVs and possible MEVs with filtered swaps values

In [ ]:
elon_weth_sim.show_swaps_and_mevs_by_token(elon_weth_filtered_swaps_df, elon_weth_mevs_df, width=15, height=5)

In [ ]:
elon_weth_sim.show_swaps_and_mevs_by_token(elon_weth_suspicious_filtered_swaps_df, elon_weth_possible_mevs_df, width=15, height=5)

### Plotting MEV transactions and possible MEV transactions with filtered swaps daily count distributions

In [ ]:
elon_weth_sim.show_swaps_and_mevs_daily_count_by_token(elon_weth_filtered_swaps_df, elon_weth_mevs_df, mevs_alter_axis=True)

In [ ]:
elon_weth_sim.show_swaps_and_mevs_daily_count_by_token(elon_weth_suspicious_filtered_swaps_df, elon_weth_possible_mevs_df, mevs_alter_axis=True)

### Plotting ratio of MEV attacks to filtered swaps

In [ ]:
elon_weth_sim.show_mevs_to_swaps_ratio(elon_weth_filtered_swaps_df, elon_weth_mevs_df)

In [ ]:
elon_weth_sim.show_mevs_to_swaps_ratio(elon_weth_suspicious_filtered_swaps_df, elon_weth_possible_mevs_df)

### Plotting ratio of MEVs and suspicious transactions to filtered swaps

In [ ]:
elon_weth_sim.show_mevs_to_reserves_ratio(elon_weth_swaps_mitigation_off_df, elon_weth_mevs_df, width=15, height=10)
elon_weth_sim.show_mevs_values_to_reserves_ratio(elon_weth_swaps_mitigation_off_df, elon_weth_mevs_df, width=15, height=10)

### Plotting transaction values distribution of passed swaps, blocked swaps and not enough reserves swaps

In [ ]:
elon_weth_sim.plot_transactions_by_type(elon_weth_swaps_mitigation_on_df, ignore_success=False, width=15, height=7)

### Plotting mitigation comparison, slice factor distribution, reserves distributions, cumulative prices distributions, transaction frequency distributions, price impact after each transaction distributions

In [ ]:
elon_weth_sim.show_mitigation_comparison(elon_weth_swaps_mitigation_off_df, elon_weth_swaps_mitigation_on_df)
elon_weth_sim.px_hist_blocked_transactions_slice(elon_weth_swaps_mitigation_on_df)
elon_weth_sim.px_hist_blocked_transactions_difference_from_oracle(elon_weth_swaps_mitigation_on_df)
elon_weth_sim.plot_slice_distribution_and_diff_limit_line(elon_weth_swaps_mitigation_on_df)
elon_weth_sim.plot_reserves_with_and_without_mitigation(elon_weth_swaps_mitigation_off_df, elon_weth_swaps_mitigation_on_df)
elon_weth_sim.plot_cumulative_prices_with_and_without_mitigation(elon_weth_swaps_mitigation_off_df, elon_weth_swaps_mitigation_off_df, make_big_num_convert=True)
elon_weth_sim.plot_price_distribution(elon_weth_swaps_mitigation_off_df, elon_weth_swaps_mitigation_on_df, separate_plots=True)
elon_weth_sim.plot_frequency_distribution(elon_weth_swaps_mitigation_on_df)
elon_weth_sim.plot_price_impact(elon_weth_swaps_mitigation_off_df, elon_weth_swaps_mitigation_on_df, 
                                smallest_y=elon_weth_swaps_mitigation_off_df['price_diff'].min(), 
                                biggest_y=elon_weth_swaps_mitigation_off_df['price_diff'].max())

In [ ]:
elon_weth_blocked_df = elon_weth_swaps_mitigation_on_df[elon_weth_swaps_mitigation_on_df['status'] == 'BLOCKED_BY_VOLATILITY_MITIGATION']
elon_weth_blocked_mevs_df = pd.merge(elon_weth_blocked_df, elon_weth_mevs_df, on='txd')
elon_weth_blocked_mevs_df

In [ ]:
elon_weth_blocked_df = elon_weth_swaps_mitigation_on_df[elon_weth_swaps_mitigation_on_df['status'] == 'BLOCKED_BY_VOLATILITY_MITIGATION']
elon_weth_blocked_mevs_df = pd.merge(elon_weth_blocked_df, elon_weth_possible_mevs_df, on='txd')
elon_weth_blocked_mevs_df

# SQUID/WETH

In [ ]:
squid_weth_mints_mitigation_off_df

In [ ]:
squid_weth_burns_mitigation_off_df

In [ ]:
squid_weth_swaps_mitigation_off_df

In [ ]:
plot_mints(squid_weth_mints_mitigation_on_df, squid_weth_mints_mitigation_off_df, pool_name='SQUID/WETH')

## MEV transactions extraction and analysis

### MEV attacks and possible MEVs extraction

There are 136 MEV attacks performed on the pool and 8789 possible attacks

In [ ]:
squid_weth_filtered_swaps_df, squid_weth_mevs_df = squid_weth_sim.extract_filtered_and_mevs_dfs()

In [ ]:
squid_weth_suspicious_filtered_swaps_df, squid_weth_possible_mevs_df = squid_weth_sim.extract_suspicious_and_filtered_swaps_dfs()

In [ ]:
squid_weth_mevs_df = squid_weth_sim.calculate_attack_profit(squid_weth_mevs_df)

### Plotting transaction values distribution of passed swaps, blocked swaps and not enough reserves swaps

In [ ]:
squid_weth_sim.plot_transactions_by_type(squid_weth_swaps_mitigation_on_df, ignore_success=False, width=15, height=7)

### Plotting mitigation comparison, slice factor distribution, reserves distributions, cumulative prices distributions, transaction frequency distributions, price impact after each transaction distributions

In [ ]:
squid_weth_sim.show_mitigation_comparison(squid_weth_swaps_mitigation_off_df, squid_weth_swaps_mitigation_on_df)
squid_weth_sim.px_hist_blocked_transactions_slice(squid_weth_swaps_mitigation_on_df)
squid_weth_sim.px_hist_blocked_transactions_difference_from_oracle(squid_weth_swaps_mitigation_on_df)
squid_weth_sim.plot_slice_distribution_and_diff_limit_line(squid_weth_swaps_mitigation_on_df)
squid_weth_sim.plot_reserves_with_and_without_mitigation(squid_weth_swaps_mitigation_off_df, squid_weth_swaps_mitigation_on_df)
squid_weth_sim.plot_cumulative_prices_with_and_without_mitigation(squid_weth_swaps_mitigation_off_df, squid_weth_swaps_mitigation_off_df, make_big_num_convert=True)
squid_weth_sim.plot_price_distribution(squid_weth_swaps_mitigation_off_df, squid_weth_swaps_mitigation_on_df, separate_plots=True)
squid_weth_sim.plot_frequency_distribution(squid_weth_swaps_mitigation_on_df)
squid_weth_sim.plot_price_impact(squid_weth_swaps_mitigation_off_df, squid_weth_swaps_mitigation_on_df, 
                                smallest_y=squid_weth_swaps_mitigation_off_df['price_diff'].min(), 
                                biggest_y=squid_weth_swaps_mitigation_off_df['price_diff'].max())
